# Fact IoT Agregado (Horário)

Agregação horária de leituras de sensores para otimização de queries.

In [ ]:
spark.sql(f"""
CREATE OR REPLACE TABLE {fqtn(SCHEMA_GOLD, 'fact_iot_agregado')} (
    equipment_id STRING,
    sensor_type STRING,
    hour_bucket TIMESTAMP,
    date_key INT,
    avg_value DOUBLE,
    min_value DOUBLE,
    max_value DOUBLE,
    reading_count BIGINT
) USING DELTA
PARTITIONED BY (date_key, sensor_type)
""")

spark.sql(f"""
INSERT OVERWRITE {fqtn(SCHEMA_GOLD, 'fact_iot_agregado')}
SELECT 
    equipment_id,
    sensor_type,
    date_trunc('hour', reading_timestamp) as hour_bucket,
    cast(date_format(date_trunc('hour', reading_timestamp), 'yyyyMMdd') as int) as date_key,
    avg(reading_value) as avg_value,
    min(reading_value) as min_value,
    max(reading_value) as max_value,
    count(*) as reading_count
FROM {fqtn(SCHEMA_SILVER, 'iot_readings_clean')}
GROUP BY 1, 2, 3
""")